# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

# Data Exploration

In [7]:
df = pd.read_csv("ChampionMatchStats.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 80 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   championName                    4960 non-null   object
 1   assists                         4960 non-null   int64 
 2   baronKills                      4960 non-null   int64 
 3   bountyLevel                     4960 non-null   int64 
 4   consumablesPurchased            4960 non-null   int64 
 5   damageDealtToBuildings          4960 non-null   int64 
 6   damageDealtToObjectives         4960 non-null   int64 
 7   damageDealtToTurrets            4960 non-null   int64 
 8   damageSelfMitigated             4960 non-null   int64 
 9   deaths                          4960 non-null   int64 
 10  detectorWardsPlaced             4960 non-null   int64 
 11  doubleKills                     4960 non-null   int64 
 12  dragonKills                     4960 non-null   

In [8]:
target = 'win'
features = [i for i in df.columns if i not in [target]]
original_df = df.copy(deep=True)
print('\n\033[1mInference:\033[0m The Datset consists of {} features & {} samples.'.format(df.shape[1], df.shape[0]))


Inference: The Datset consists of 80 features & 4960 samples.


In [9]:
df.nunique().sort_values()

unrealKills                       1
sightWardsBoughtInGame            1
nexusTakedowns                    2
teamEarlySurrendered              2
nexusLost                         2
                               ... 
totalDamageDealtToChampions    4477
totalDamageTaken               4505
physicalDamageDealt            4652
totalDamageDealt               4756
perks                          4830
Length: 80, dtype: int64

In [10]:
nu = df[features].nunique().sort_values()
nf = []; cf = []; nnf = 0; ncf = 0; #numerical & categorical features

for i in range(df[features].shape[1]):
    if nu.values[i]<=16:cf.append(nu.index[i])
    else: nf.append(nu.index[i])

print('\n\033[1mInference:\033[0m The Datset has {} numerical & {} categorical features.'.format(len(nf),len(cf)))


Inference: The Datset has 47 numerical & 32 categorical features.


In [11]:
display(df.describe())

,assists,baronKills,bountyLevel,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,...,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced
count,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000,...,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000,4960.0,4960.000000,4960.000000,4960.000000,4960.000000
mean,10.812702,0.067137,0.955040,3.542540,2137.051815,7735.664516,2137.051815,17377.822581,5.927621,1.766331,...,1035.740927,1089.584476,0.805444,1.688508,4.175403,0.0,20.243347,2.041331,2.654234,8.400403
std,9.417208,0.267423,1.919271,3.327188,2685.396208,11046.399576,2685.396208,16555.384009,3.722557,2.512810,...,1692.841865,1341.458376,1.184634,1.794535,3.308053,0.0,21.539289,2.842953,3.217968,10.197849
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,4.000000,0.000000,0.000000,1.000000,247.000000,940.000000,247.000000,7278.250000,3.000000,0.000000,...,61.750000,298.750000,0.000000,0.000000,2.000000,0.0,5.000000,0.000000,0.000000,1.000000
50%,8.000000,0.000000,0.000000,3.000000,1173.000000,3039.000000,1173.000000,12338.000000,6.000000,1.000000,...,518.500000,656.500000,0.000000,1.000000,4.000000,0.0,15.000000,1.000000,2.000000,6.000000
75%,16.000000,0.000000,1.000000,5.000000,3020.000000,9726.250000,3020.000000,21634.750000,8.000000,3.000000,...,1220.000000,1353.000000,1.000000,3.000000,7.000000,0.0,27.000000,3.000000,4.000000,10.000000
max,65.000000,2.000000,16.000000,25.000000,21896.000000,94556.000000,21896.000000,203932.000000,25.000000,20.000000,...,18412.000000,16272.000000,9.000000,10.000000,11.000000,0.0,161.000000,23.000000,35.000000,89.000000
